# Physical Object: Valve

The `Valve` class models a controllable valve. Its operation is very similar to a `Gate`, calculating outflow based on the head difference and its opening percentage. The opening is controlled by a `control_signal` which can range from 0 (fully closed) to 100 (fully open).

### State Variables
- `opening` (float): The current opening of the valve (0-100%).
- `outflow` (float): The calculated outflow through the valve (m^3/s).

### Parameters
- `discharge_coefficient` (float): A dimensionless factor C_d used in the flow calculation.
- `diameter` (float): The diameter of the valve's pipe (m).

## Simulation Example

The following example simulates a valve being commanded to different opening percentages. We can observe how the outflow directly responds to these commands, given a constant head difference.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.valve import Valve
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(opening=0.0)
parameters = Parameters(
    discharge_coefficient=0.8,
    diameter=0.5
)

# 2. Create a Valve instance
valve = Valve(name="control_valve", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 1.0
simulation_duration = 100
num_steps = int(simulation_duration / dt)
history = []
target_openings = []

# 4. Run the simulation loop
for t in range(num_steps):
    # Create a varying control signal for the target opening
    if t < 20:
        target = 75.0
    elif t < 60:
        target = 25.0
    else:
        target = 100.0
    target_openings.append(target)
    
    action = {
        'control_signal': target,
        'upstream_head': 20.0, # Constant upstream water level
        'downstream_head': 10.0  # Constant downstream water level
    }
    
    current_state = valve.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Valve simulation complete.")

## Results and Visualization

The plots show that the valve's `opening` state instantly matches the `control_signal`. The `outflow` is directly proportional to this opening, as expected from the orifice equation.

In [ ]:
# Extract data from history
time = [i * dt for i in range(num_steps)]
openings = [h['opening'] for h in history]
outflows = [h['outflow'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Target Opening (%)': target_openings,
    'Actual Opening (%)': openings,
    'Outflow (m^3/s)': outflows
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Valve Opening
ax1.plot(df['Time'], df['Actual Opening (%)'], label='Valve Opening', color='blue', drawstyle='steps-post')
ax1.set_ylabel('Opening (%)')
ax1.set_title('Valve Dynamics')
ax1.grid(True)
ax1.legend()

# Outflow
ax2.plot(df['Time'], df['Outflow (m^3/s)'], label='Outflow', color='green')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()